We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can encirh this info with spotify data (because it only allows to 50 last tracks in a beta api)
After that we can try som simple ML models on it :D 

In [10]:
import requests as re
#import numpy as np
import time
import pandas as pd
import datetime
from datetime import date, timedelta
import json
import requests_cache

### Pre-work

In [11]:
#Get the csv from the last request
#COLUMNS 0 -> BAND NAME
#COLUMNS 1 -> SONG NAME
#COLUMNS 2 -> PLAYED AT UNIX TIMESTAMP
tracks = pd.read_csv('played_tracks.csv')

In [12]:

tracks.drop(tracks.columns[2], axis = 1, inplace = True)
#clear duplicate columns
tracks = tracks.drop_duplicates()
#rename columns
tracks.rename(mapper= {'0': 'band_name', '1': 'song_name'}, axis = 1, inplace = True)

## Spotify API

In [13]:
#create a request cache
requests_cache.install_cache('track_features_cache')

In [14]:
#song id
#request.json()['tracks']['items'][0]['id']
#artist
#request.json()['tracks']['items'][0]['album']['artists'][0]['name']

In [29]:
#define a function for the api request that returns the json response
def search_song_request(_song_name):
    params = {'q' : _song_name, 'type': 'track', 'limit': 50}
    r = re.get('https://api.spotify.com/v1/search', headers = headers, params = params)
    return r.json()['tracks']['items']

#define a function to make the api request, iterate on the response and get the song id based on the artist
def get_song_id(_song_name, _band_name):
    _response = search_song_request(_song_name)
    for i in range(len(_response)):
        #there must be a better way of doing this byt it'll work for now: we iterate through the entire artist list - there is a lot of useless data in the response
        for j in range(len(_response[i]['album']['artists'])):
            if _response[i]['album']['artists'][j]['name'] == _band_name:
                return _response[i]['id']
            else:
                pass
    #if no returns, we could not find id, so it returns not_found
    return 'not_found'


In [16]:
#Authentication - getting an access token
auth_response = re.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials',
    'client_id': '6cb782c1b0404843b3e5a06e8361cb6e',
    'client_secret': 'a32c28deab714fa89091395089cbca90',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#create a header with the access token
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [32]:
#generate a test df to iterate:
df_test = tracks.sample(10)

In [33]:
#print("start:", datetime.datetime.now())
df_test['sp_id'] = df_test.apply(lambda x: get_song_id(x['song_name'], x['band_name']), axis = 1)
#print("end:", datetime.datetime.now())
df_test

,band_name,song_name,sp_id
3917,Ben Klock,Gooldy Sin (feat. Elif Biçer) - Original Mix,6DnCRZW2NCaPZmkPoXzRBB
2746,The Hellacopters,Carry Me Home,4BHxhxzEjPivA5ho9Xqtw4
4796,Earl Sweatshirt,Pre (feat. SK La' Flare),4B31QOL13iBeuIsaGD20BS
9588,Ross From Friends,Back Into Space,not_found
7217,Stephan Bodzin,Zulu - Agents of Time Remix,2yOY1ZFVyYV6FopyzhYA6z
1368,Leon Vynehall,Butterflies,not_found
9120,Miles Davis,Two Bass Hit,0DtEf3aHfOgaCFJQO98x0c
8176,Motörhead,Overkill - Live in England 1981,5F1PWC6InPSZhhzkkvnUME
2116,The Gun Club,Sex Beat,not_found
3184,Plaid,Meds Fade,1xdnBrz6S6fC4kQPJllb68


In [19]:
x = search_song_request('One More Day')

In [28]:
x[7]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4D75GcNG95ebPtNvoNVXhz'},
    'href': 'https://api.spotify.com/v1/artists/4D75GcNG95ebPtNvoNVXhz',
    'id': '4D75GcNG95ebPtNvoNVXhz',
    'name': 'Afrojack',
    'type': 'artist',
    'uri': 'spotify:artist:4D75GcNG95ebPtNvoNVXhz'}],
  'available_markets': ['AD',
   'AE',
   'AL',
   'AR',
   'AU',
   'BA',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'BY',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HR',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'KZ',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MD',
   'ME',
   'MK',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   